In [50]:
from subtitle_process import fetch_subtitle
import getpass
import os
import json
import re
from templates import *
from functions import inference

from langchain.prompts import PromptTemplate

In [62]:
# 设置临时环境变量
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("bilibili_ssid")

In [63]:
with open("url.json", 'r') as file:
    data = json.load(file)

In [65]:
problems = []
for bv in data:
    problems.append(fetch_subtitle(bv))

Extracted segment: 1Xr4y157pm
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/7717160297938316452d7636c209b867070aa4f39a6575129b?auth_key=1720597487-a6b1d58c60674ec3b60ad6cfc09961ed-0-781c6e27cdf639842e739cf80d0382fb
Extracted segment: 1wW4y1H73J
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/9438733228534792094338de9bc70d602e8533ae91d5246a33?auth_key=1720597492-2d51ce06b6e14e3b81679acdbfe32d7e-0-a8e48f0c288410d4e253465752a76b09
Extracted segment: 1A84y1b76v
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/606291503923343443061518d560dd161c9dc56f4a83c61596?auth_key=1720597496-bf269606c1014b04b0431b0dadd983cf-0-808ba2d2bdac0db12fe58cea2e7a2988
Extracted segment: 1hK411Z7YE
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/475217265893072095f41494bc9d3315303

In [66]:
def extract_question_explanations(responses):
    questions_and_explanations = []
    for response in responses:
        pattern = re.compile(r"### 原题：\n(.+?)\n\n### 讲解：\n(.+?)\n\n### 相似题一：\n(.+?)\n\n### 相似题一讲解：\n(.+?)(?:\n\n|$)", re.DOTALL)
        matches = pattern.findall(response)

        for match in matches:
            questions_and_explanations.append({
                "original_question": match[0].strip(),
                "original_explanation": match[1].strip(),
                "similar_question_1": match[2].strip(),
                "similar_explanation_1": match[3].strip(),
            })

    with open('casestudy_problem_answer.json', 'w', encoding='utf-8') as f:
        json.dump(questions_and_explanations, f, ensure_ascii=False, indent=4)


In [67]:
template_sub_1_similar = r"""
以下是一段视频字幕，请提取出题目和老师对题目的讲解，同时构造一个相似题目并生成与老师讲解风格相似的讲解。请按如下格式输出：

### 原题：
（提取出的原题）

### 讲解：
（提取出的讲解）

### 相似题一：
（构造的相似题目一）

### 相似题一讲解：
（构造的相似题目一的讲解）

视频字幕：
{subtitle}

注意事项：
1. 对于每一道原题，仅构造一道相似题，对于每一题都当作是新题来讲解，不要出现任何类似于“根据我们刚才说的”这样的话，每一个讲解必须以答案结尾，不能够没有答案。
2. 保留老师讲解风格但是把和讲解无关的内容过滤掉，比如自问自答的部分都要删除掉。
"""

template_expl = r"""
    Role:
        你是一位专业且循循善诱数学教师名为小派老师,可以解决小学、初中、高中的数学题目,你将一对一的讲解一个题目。 
    Goals:
        根据给定的板书内容像老师一样讲解板书内容，根据输入决定你正在输出的是讲解几，比如如果输入是板书一，那么输出就是讲解一，以此类推。

    严格按照以下列字符串格式输出:
        ### 讲解一:    ### 讲解二:      ...

    原题和参考信息:
        题目:{question}
        参考信息:{ref}

    注意事项:
        1. 你的讲解要尽量贴合人类老师的讲解风格，言简意赅，清晰明了。
        2. 讲解内容应针对板书的具体步骤进行详细说明，解释每一步的思路和方法。
        3. 确保每一个讲解都对应一个板书。
        4. 板书和讲解的编号必须一一对应，格式必须严格遵守。
        5. 讲解内容应突出解题思路和方法，避免过多冗余信息。
        6. 你的讲解对象是一位学生而不是一群学生，一定不要出现同学们这样的复数词汇
        7. 其它注意事项：
            a: 初中不等式还没有学到用方括号圆括号以及并集交集等表示范围，避免使用这些表达。
            b: 初中几何题目如证明全等三角形，正面直线平行，板书中需要写上使用了什么定理，并且注意写法是定理短句加括号写在式子后面。
            c: 对于选择题的问题，如果需要把各个选项铺陈，用ABCD作为列表标题，别用括号1234。
            d: 初中还没有学过排列组合的问题，避免使用排列组合的知识点解答问题。
            e: 对于所有题目尽可能有最简单的方式来进行解答，以最大限度上避免超纲的情况发生。

    下面是板书的内容(请确保每一个讲解都对应一个板书)：
    {notes}

    下面是一些名师讲解的其他题目，请你学习他们的讲解风格，并运用到板书的讲解中(注意：你的讲解对象是一位学生而不是一群学生，一定不要出现同学们这样的复数词汇)。
    注意：初中还没有学过排列组合的问题，避免使用排列组合的知识点解答问题。
    {model_expl}
"""

template_orig = r"""
    Role:
    你是一位专业且循循善诱数学教师名为小派老师,可以解决小学、初中、高中的数学题目,你将一对一讲解一个题目
    Goals:
    根据给定信息按照要求像老师一样讲解一个题目

    技能
    可以使用代码辅助解题
    能够处理各种计算

    Workflows:
    你会按下面的框架来讲解题目
        1. 给出该题目涉及的知识点
        2. 判断给出的信息中是不是包含原题并且有原题的解答过程，如果有原题，板书使用原题的解答步骤，并对应生成讲解；如果没有原题，则参照相似题的思路和解法给出结果

    第一部分:原题和参考信息
        题目:{question}
        学段：初中，你需要适配初中的知识点，解法和书写表达不超纲
        参考信息:{ref}
    第二部分: 以下列字符串格式输出
        知识点讲解:这题涉及到了一些xxx相关的知识点，通常使用xxx方法解决 ### 板书一:     ### 板书二:       ### 讲解一:       ### 讲解二:      ...
    注意事项:
        1.你擅将利用参考信息来解答问题,如果有参考信息，参考信息是这个题目的相似题，你需要参考其解题思路与步骤。
        2.你的解题思路、方法、讲解需要尽可能地参照参考信息,在计算部分使用代码进行辅助，减少自己解题的比重。
        3.知识点讲解部分，需给出题目涉及的知识点考点、题目解题的关键点，必须使用模板，有且只有这一句话就行：本题考查的知识点是.....，解题关键点是......。

        4.板书是这个题目的答题步骤，思路和知识点讲解部分做到一致，按照标准答案的书写方式去写，板书切分块的模式依照 条件+结论为一个单元分块，板书一/二/三/....整个解题步骤不超过十个板书+讲解单元；讲解是对应板书模块的说明。在板书的最开始按需写出"解："或"证明："，在板书最后写出"答："或"故答案是："。
        5.每个式子要独立成行，不输出连续很长的一个式子。
        6.注意板书只写计算公式,尽量少出现中文,一些推导过程涉及到详细的题目条件的要用 依题意得/由题可知 等说法省略详细条件，避免板书中出现很长的中文句子内容，尽量用式子说话, 式子计算过程要做到关键步骤不省略不跳步,如果需要,在一些公式前使用$\because$ $\therefore$等表达因为所以的数学符号
        7.禁止出现**表示标题和-表示列表这种markdown格式的语法，latex公式中避免使用\text{{}}除非必须，适时换行，不要用\\表示换行。
        8.如果题目包含多个小题，板书中对应的开始的地方展示小题题号，比如 (1)，(2)，(3)，禁止重复展示小标题序号，比如板书一板书二都是对于(1)小题的解答，则只需要在板书一开始写一次(1)，板书二不能再写一次(1)。
        9.讲解和板书一一对应，讲解要重点强调对应板书涉及到的思路和方法，讲解要抓住题目重点，言简意赅，语言精炼。  
        例如:
        "知识点讲解:本题考查的知识点是.....，解题关键点是......。### 板书一: $\because$ $\frac{{x^2-2*x-1}}{{x-1}}=\frac{{(x-1)^2}}{{x-1}}=(x-1)^{{2-1}}=x-1$### 板书二: $\therefore$ $\frac{{x^2-2*x-1}}{{x-1}}*(x-2)=(x-1)(x-2)=x^2-3*x+2$ \n故答案是$x^2-3*x+2$### 讲解一:首先利用完全平方公式将分子化简,随后发现分子分母都是(x-1)的次幂,因此将指数相减即可。### 讲解二:根据上述化简结果,代入原式即可得到最终答案$x^2-3*x+2$"
        10.返回的字段中只能出现知识点讲解、板书N、讲解N(N代表中文里的一、二、三等正整数)，且所有的板书模块出完了才能出讲解模块，板书模块数量和讲解模块数量必须一致，内容必须一一对应...不能出任何现其他的字段，在最后一个板书部分要有总结性话语，在这个例子中板书二是最后一个板书部分，所以最后一句话说 故答案是...，你的回答中最后一个板书n的部分中，最后一句话必须是，故答案是...
        11.板书参考书写范式，你需要根据题型参照范式输出板书：
            {ref_form}
        12.其它注意事项：
            a: 初中不等式还没有学到用方括号圆括号以及并集交集等表示范围，避免使用这些表达。
            b: 初中几何题目如证明全等三角形，正面直线平行，板书中需要写上使用了什么定理，并且注意写法是定理短句加括号写在式子后面。
            c: 对于选择题的问题，如果需要把各个选项铺陈，用ABCD作为列表标题别用括号1234。
            d: 对于所有题目尽可能用解释性的语言进行解答，不到万不得已不用公式直接求解，以最大限度上避免超纲的情况发生。
        13.板书不能超过10条，如果10条写不下，使用别的方法来解题。
"""

In [68]:
raw_problem_answer = []
for problem in problems:
    raw_problem_answer.append(await inference({"subtitle": problem}, template_sub_1_similar))


executing
### 原题：
已知AC等于AB，角ACB等于90度，BD垂直于CD于点D，BD等于2，AD等于4倍根号2，求CB的长度。

### 讲解：
我们发现CB在直角三角形ABC当中，这个三角形ABC是等腰直角三角形，但是我们不知道线段AB和AC的长度，所以无法直接求出线段CB的长度。我们发现线段CB除了在三角形ABC当中之外，还在三角形BCD当中。在这个三角形当中，我们仅仅知道线段BD的长度，而不知道线段CD的长度，所以依然无法求出线段BC的长度。

显然这个问题需要辅助线。我们发现有两个直角，这两个直角所对的一条线段BC都对着线段BC。当两个直角对着一条定线段时，也就是说定角定弦有引援，所以这是一个引圆问题。圆心一定在BC的中点上。做出这个圆之后，ABCD四点共圆。

在RT3角形BCD当中，我们仅仅知道一条线段BD的长度，依然无法求出线段AB的长度。做出这个圆之后，我们发现角ABC是45度角，则角ADC也是45度角，因为在同圆或等圆中同弧或等弧所对的圆周角是相等的。AC弧所对的圆周角是ABC，它们是相等的，都是45度。

当我们知道角ADC是45度时，角CDB是90度，则角BDA是135度。当我们遇到135度时，我们通常在它的外面做一个直角三角形，这个直角三角形一定是等腰直角三角形。延长BD，然后过A点做BD延长线的垂线，形成一个直角三角形。

在这个直角三角形当中，AD长是4倍的根号2，线段D长和线段A1长都是4。在直角三角形ADE当中，N1长是4，D长也是4，而BD长是2，那么B1长就是6。在直角三角形ABE当中，我们利用勾股定理可以求出线段AB的长度。AB长求出来了乘以根号2，那就是线段CB的长度。CB长等于AB长的根号2倍，而AB长在直角三角形ABE当中，我们利用勾股定理求得4的平方加上6的平方等于2倍的根号13，再开平方等于2倍根号13。2倍根号13再乘以根号2，线段BC长就是2倍的根号26。

### 相似题一：
已知AC等于AB，角ACB等于90度，BD垂直于CD于点D，BD等于3，AD等于3倍根号2，求CB的长度。

### 相似题一讲解：
我们发现CB在直角三角形ABC当中，这个三角形ABC是等腰直角三角形，但是我们不知道线段AB和AC的长度，所以无法直接求出线段CB的长度。我们发现线段CB除了在三角形ABC当中之外，还在三角形

In [70]:
extract_question_explanations(raw_problem_answer)

In [71]:
with open("casestudy_problem_answer.json", 'r') as file:
    data = json.load(file)

[{'original_question': '已知AC等于AB，角ACB等于90度，BD垂直于CD于点D，BD等于2，AD等于4倍根号2，求CB的长度。',
  'original_explanation': '我们发现CB在直角三角形ABC当中，这个三角形ABC是等腰直角三角形，但是我们不知道线段AB和AC的长度，所以无法直接求出线段CB的长度。我们发现线段CB除了在三角形ABC当中之外，还在三角形BCD当中。在这个三角形当中，我们仅仅知道线段BD的长度，而不知道线段CD的长度，所以依然无法求出线段BC的长度。\n\n显然这个问题需要辅助线。我们发现有两个直角，这两个直角所对的一条线段BC都对着线段BC。当两个直角对着一条定线段时，也就是说定角定弦有引援，所以这是一个引圆问题。圆心一定在BC的中点上。做出这个圆之后，ABCD四点共圆。\n\n在RT3角形BCD当中，我们仅仅知道一条线段BD的长度，依然无法求出线段AB的长度。做出这个圆之后，我们发现角ABC是45度角，则角ADC也是45度角，因为在同圆或等圆中同弧或等弧所对的圆周角是相等的。AC弧所对的圆周角是ABC，它们是相等的，都是45度。\n\n当我们知道角ADC是45度时，角CDB是90度，则角BDA是135度。当我们遇到135度时，我们通常在它的外面做一个直角三角形，这个直角三角形一定是等腰直角三角形。延长BD，然后过A点做BD延长线的垂线，形成一个直角三角形。\n\n在这个直角三角形当中，AD长是4倍的根号2，线段D长和线段A1长都是4。在直角三角形ADE当中，N1长是4，D长也是4，而BD长是2，那么B1长就是6。在直角三角形ABE当中，我们利用勾股定理可以求出线段AB的长度。AB长求出来了乘以根号2，那就是线段CB的长度。CB长等于AB长的根号2倍，而AB长在直角三角形ABE当中，我们利用勾股定理求得4的平方加上6的平方等于2倍的根号13，再开平方等于2倍根号13。2倍根号13再乘以根号2，线段BC长就是2倍的根号26。',
  'similar_question_1': '已知AC等于AB，角ACB等于90度，BD垂直于CD于点D，BD等于3，AD等于3倍根号2，求CB的长度。',
  'similar_explanation_1': '我们发现CB在直角三角形ABC当中，这个三角形ABC是等腰

In [72]:
model_explanation = ""
for ps in data[:3]:
    model_explanation += "题目:\n" + ps['original_question'] + '\n\n'
    model_explanation += "讲解:\n" + ps['original_explanation'] + '\n\n'
    # model_explanation += "题目:\n" + ps['similar_question_1'] + '\n\n'
    # model_explanation += "讲解:\n" + ps['similar_explanation_1'] + '\n\n'
model_explanation

'题目:\n已知AC等于AB，角ACB等于90度，BD垂直于CD于点D，BD等于2，AD等于4倍根号2，求CB的长度。\n\n讲解:\n我们发现CB在直角三角形ABC当中，这个三角形ABC是等腰直角三角形，但是我们不知道线段AB和AC的长度，所以无法直接求出线段CB的长度。我们发现线段CB除了在三角形ABC当中之外，还在三角形BCD当中。在这个三角形当中，我们仅仅知道线段BD的长度，而不知道线段CD的长度，所以依然无法求出线段BC的长度。\n\n显然这个问题需要辅助线。我们发现有两个直角，这两个直角所对的一条线段BC都对着线段BC。当两个直角对着一条定线段时，也就是说定角定弦有引援，所以这是一个引圆问题。圆心一定在BC的中点上。做出这个圆之后，ABCD四点共圆。\n\n在RT3角形BCD当中，我们仅仅知道一条线段BD的长度，依然无法求出线段AB的长度。做出这个圆之后，我们发现角ABC是45度角，则角ADC也是45度角，因为在同圆或等圆中同弧或等弧所对的圆周角是相等的。AC弧所对的圆周角是ABC，它们是相等的，都是45度。\n\n当我们知道角ADC是45度时，角CDB是90度，则角BDA是135度。当我们遇到135度时，我们通常在它的外面做一个直角三角形，这个直角三角形一定是等腰直角三角形。延长BD，然后过A点做BD延长线的垂线，形成一个直角三角形。\n\n在这个直角三角形当中，AD长是4倍的根号2，线段D长和线段A1长都是4。在直角三角形ADE当中，N1长是4，D长也是4，而BD长是2，那么B1长就是6。在直角三角形ABE当中，我们利用勾股定理可以求出线段AB的长度。AB长求出来了乘以根号2，那就是线段CB的长度。CB长等于AB长的根号2倍，而AB长在直角三角形ABE当中，我们利用勾股定理求得4的平方加上6的平方等于2倍的根号13，再开平方等于2倍根号13。2倍根号13再乘以根号2，线段BC长就是2倍的根号26。\n\n题目:\n已知A减B等于九，求A平方减去B平方减去18倍B。\n\n讲解:\n这道题难度不大，大家一定要会。求解这道题的方法比较多，这里分享几种。\n\n第一种方法：因为我们要求的这个式子A平方减去B平方，这不是一个平方差公式，所以原式等于(A加B)乘以(A减B)再减去18倍B。又知道A减B等于九，对原式就等于九倍的(A加B)减去18倍B。等于九倍A

In [73]:
orig_answers = []
orig_notes = []
orig_expls = []
refined_explanation = []
questions = []

In [74]:
'''
对于每一个<原题,相似题>，用两个prompt来分别对相似题进行解答
    1. 用original prompt对相似题直接进行解答
    2. 将原题解答及相似题解答作为few shots examples喂给另一个prompt来对比两种的效果哪一个比较好
'''

with open("casestudy_problem_answer.json", 'r') as file:
    problem_set = json.load(file)


for ps in problem_set:
    orig_answers.append(await inference({"question": ps['similar_question_1'], "ref_form": ref_form, "ref": ref}, template_orig, max_tokens=2000))
    pattern1 = r"### 板书一:[\s\S]*?(?:### 板书[二三四五六七八九十]+:[\s\S]*?)*?(?=### 讲解一)"
    orig_note = re.search(pattern1, orig_answers[-1]).group()
    pattern2 = r"### 讲解一:[\s\S]*"
    orig_expl = re.search(pattern2, orig_answers[-1]).group()
    orig_notes.append(orig_note)
    orig_expls.append(orig_expl)
    questions.append(ps['similar_question_1'])
    refined_explanation.append(await inference({"question": ps['similar_question_1'], "ref": ref, "notes": orig_note, "model_expl": model_explanation}, template_expl, max_tokens=2000))

# print(orig_notes)
# print(refined_explanation)    

executing
知识点讲解:本题考查的知识点是直角三角形的性质和勾股定理，解题关键点是利用直角三角形的性质和勾股定理求解边长。

### 板书一:
$\because \angle ACB = 90^\circ$
$\therefore \triangle ACB$ 是直角三角形

### 板书二:
$\because AC = AB$
$\therefore \triangle ACB$ 是等腰直角三角形

### 板书三:
$\because BD \perp CD$ 于点D
$\therefore \triangle BCD$ 是直角三角形

### 板书四:
$AD = 3\sqrt{2}$

### 板书五:
$BD = 3$

### 板书六:
$\because \triangle ABD$ 是直角三角形
$\therefore AB = \sqrt{AD^2 + BD^2}$
$AB = \sqrt{(3\sqrt{2})^2 + 3^2}$
$AB = \sqrt{18 + 9}$
$AB = \sqrt{27}$
$AB = 3\sqrt{3}$

### 板书七:
$\because AC = AB$
$\therefore AC = 3\sqrt{3}$

### 板书八:
$\because \triangle ACB$ 是等腰直角三角形
$\therefore CB = AC \cdot \sqrt{2}$
$CB = 3\sqrt{3} \cdot \sqrt{2}$
$CB = 3\sqrt{6}$

### 讲解一:
首先，根据题意，$\angle ACB = 90^\circ$，所以$\triangle ACB$是直角三角形。

### 讲解二:
由于$AC = AB$，所以$\triangle ACB$是等腰直角三角形。

### 讲解三:
根据题意，$BD \perp CD$于点D，所以$\triangle BCD$是直角三角形。

### 讲解四:
题目给出$AD = 3\sqrt{2}$。

### 讲解五:
题目给出$BD = 3$。

### 讲解六:
因为$\triangle ABD$是直角三角形，所以可以用勾股定理求出$AB$的长度：
$AB = \sqrt{AD^2 + BD^2} = \sqrt{(3\sq

In [75]:
with open("compare_results_p9.md", 'a') as file:
    for i in range(len(questions)):
        file.write("# 原题：\n" + questions[i] + '\n\n')
        file.write("# 原本板书：\n" + orig_notes[i] + '\n\n')
        file.write("# 原本解答：\n" + orig_expls[i] + '\n\n')
        file.write("# 新解答：\n" + refined_explanation[i] + '\n\n')